In [1]:
import numpy as np
from functools import reduce
import functions as fn
from matplotlib import pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from tqdm import tqdm
from matplotlib.text import Text
from circuit_obj import Circuit
from itertools import product

from matplotlib import cm
from matplotlib.colors import LogNorm

from scipy.special import comb

import warnings

# 1) Globally ignore any warning whose text matches “too many iterations”
warnings.filterwarnings("ignore", message=".*too many iterations.*")

# from joblib import Parallel, delayed
colors = ["#394F87", "#A0B2E8", "#F0BCC1", "#7F0E0E"]
cmap1 = LinearSegmentedColormap.from_list("custom_cmap", colors, N=256)
norm = plt.Normalize(vmin=0, vmax=np.pi)
smJ = plt.cm.ScalarMappable(cmap=cmap1, norm=norm)

256


In [2]:

import numpy as np
from itertools import combinations

def spin_basis_1d(N, m=0.0, dtype=np.uint32, tol=1e-8):
    """
    NumPy‐only re‑implementation of quspin.basis.spin_basis_1d
    valid for any N (even or odd) and allowing half‑integer total M.

    Parameters
    ----------
    N : int
        Number of spin‑1/2 sites.
    m : float
        Magnetization per site; 2*m*N must be (nearly) integer.
    dtype : np.dtype
        Output integer dtype for the bit‑strings.
    tol : float
        Tolerance for floating‑point checks.

    Returns
    -------
    Ns : int
        Number of basis states = binomial(N, n_up).
    states : ndarray[int]
        Unsigned ints (dtype) whose binary representation encodes the spin configuration,
        sorted descending exactly as QuSpin.spin_basis_1d does.
    """

    # 1) total 2M = 2*m*N must be integer
    twoM_float = 2 * m * N
    if abs(twoM_float - round(twoM_float)) > tol:
        raise ValueError(f"2*m*N = {twoM_float} not (nearly) integer")
    twoM = int(round(twoM_float))

    # 2) number of up‑spins: n_up = (2M + N) / 2
    if (twoM + N) % 2 != 0:
        # Should never happen if twoM is int and N is int, but just in case
        raise ValueError(
            f"(2*m*N + N)/2 = {(twoM+N)/2} not integer"
        )
    n_up = (twoM + N)//2
    if not (0 <= n_up <= N):
        raise ValueError(f"n_up = {n_up} out of range [0, {N}]")

    # 3) enumerate all combinations of n_up ones in N bits,
    #    in descending‐bit lex order to match QuSpin exactly.
    states_list = []
    # iterate over positions N-1, N-2, …, 0
    for comb_ in combinations(range(N-1, -1, -1), n_up):
        s = 0
        for i in comb_:
            s |= (1 << i)
        states_list.append(s)
        
    states = np.array(states_list, dtype=dtype)
    assert len(states) == comb(N, int(N/2 + m * N)), \
        f'Wrong basis for {N}, {m}: {len(states)} != {comb(N, int(N/2 + m * N))}'

    return states

In [5]:
N = 20

# Circuit parameters
T = 1000
circuit_realizations = 2
symmetry = ['U1', 'SU2', 'Z2', 'ZK'][0]
geometry = ['random', 'brickwork'][1]
alphaT = 1 # Parameter to slice the circuit thinner

# Mask parameters
sites_to_keep = range((N-8), N)
qkeep = np.array(sites_to_keep)
qthrow = np.arange(N)[np.isin(np.arange(N), qkeep, invert=True)]
Ns = len(sites_to_keep)

Ne = N - Ns
projectors_s, U_U1_s = fn.build_projectors(Ns) 
projectors_e, U_U1_e = fn.build_projectors(Ne)
sectors_se = np.array([int(comb(N, int(N/2+m))) for m in np.arange(-N/2, N/2+1)])
sectors_s = np.array([int(comb(Ns, int(Ns/2+m))) for m in np.arange(-Ns/2, Ns/2+1)])
sectors_e = np.array([int(comb(Ne, int(Ne/2+m))) for m in np.arange(-Ne/2, Ne/2+1)])

masks_dict = fn.load_mask_memory(N, (2 if symmetry!='ZK' else K))

alphas = np.array([1]) # np.r_[np.linspace(0.1, 2, 10)]
qthrow = np.arange(N)[np.isin(np.arange(N), sites_to_keep, invert=True)]
alphas, sites_to_keep

(array([1]), range(12, 20))

In [ ]:
def initial_state_test(target_sector):
    '''
    Create the 
    l state as (R_y(theta) |J_max>_Ns) tensor (|J_max>_Ne)
    '''
    assert target_sector <= Ns, "target_sector must be less than/eq the number of qubits in the system"
    
    # system:
    state_s = np.zeros(2**(len(qkeep)), dtype=np.complex128)
    cum_sectors_s = np.cumsum(sectors_s)
    
    if target_sector == int(target_sector):
        tgt = int(target_sector)
        element = cum_sectors_s[tgt-1] if tgt > 0 else 0
        element += sectors_s[tgt]//2
        state_s[element] = 1
        state_s[-element-1] = 1
    else:
        tgt = int(np.ceil(target_sector))
        element = cum_sectors_s[tgt-1] if tgt > 0 else 0
        element += sectors_s[tgt]//2
        state_s[element] = 1
        tgt = int(np.floor(target_sector))
        element = cum_sectors_s[tgt-1] if tgt > 0 else 0
        element += sectors_s[tgt]//2
        state_s[-element-1] = 1
    
    state_s /= np.linalg.norm(state_s)
    state_s = U_U1_s @ state_s
        
    #environment:
    state_e = np.zeros(2**(len(qthrow)), dtype=np.complex128)
    
    if not Ne % 2: # even
        size_zero_sector = sectors_e[Ne//2]
        start = (2**Ne - size_zero_sector)//2
        end = start + size_zero_sector
        state_e[start:end] = 1
    else:
        size_zero_sector = sectors_e[Ne//2]
        start = (2**Ne - 2*size_zero_sector)//2
        end = start + size_zero_sector
        state_e[start:end] = 1
        
    state_e = U_U1_e @ state_e
    
    state_e /= np.linalg.norm(state_e)

    return np.kron(state_e, state_s)

In [ ]:
thetas = np.array([.3, .4, .5])*np.pi # theta


fig2, axs = plt.subplots(1, len(thetas), figsize=(16, 8))
fig3, ax1s = plt.subplots(1, len(thetas), figsize=(16, 8))
    
renyis = []
for idx, theta in enumerate(thetas):
    # state = initial_state_test(theta)
    state = fn.initial_state(N, sites_to_keep, theta, 'homogenous')
    
    rho_s = fn.ptrace(state, sites_to_keep)
    rho_s_U1 = U_U1_s.conj().T @ (rho_s) @ U_U1_s
    
    rho_s_tw = fn.manual_U1_tw(rho_s, projectors_s)
    rho_s_tw_U1 = U_U1_s.conj().T @ (rho_s_tw) @ U_U1_s
    
    cmap = cm.plasma.copy()
    cmap.set_bad(color='white')  # or 'none' for full transparency
    
    d1 = np.ma.masked_less(np.abs(rho_s_U1), 1e-12)
    im = axs[idx].imshow(d1, cmap=cmap, norm=LogNorm(1e-12, 1e0), interpolation='none') 
    
    d1 = np.ma.masked_less(np.abs(rho_s_tw_U1), 1e-12)
    im = ax1s[idx].imshow(d1, cmap=cmap, norm=LogNorm(1e-12, 1e0), interpolation='none') 
    
    rho_modes = fn.asymmetry_modes(rho_s_U1, sectors_s)
    print('SYS: ', [f'{omega_idx}: {fn.compute_norm(rho_omega):.2f}' for omega_idx, rho_omega in enumerate(rho_modes)])
    
    renyis.append((fn.renyi_divergence_sym(rho_s, symmetry, 1).real))
[*zip(thetas, renyis)]


In [ ]:
if Ne < 12:
    
    fig2, axs = plt.subplots(1, 2, figsize=(16, 8))

    state = initial_state_test(theta)

    rho_e = fn.ptrace(state, qthrow)

    rho_e_tw = fn.manual_U1_tw(rho_e, projectors_e)

    cmap = cm.plasma.copy()
    cmap.set_bad(color='white')  # or 'none' for full transparency

    d1 = np.ma.masked_less(np.abs(U_U1_e.conj().T @ (rho_e) @ U_U1_e), 1e-12)
    im = axs[0].imshow(d1, cmap=cmap, norm=LogNorm(1e-12, 1e0), interpolation='none') 

    d1 = np.ma.masked_less(np.abs(U_U1_e.conj().T @ (rho_e - rho_e_tw) @ U_U1_e), 1e-12)
    im = axs[1].imshow(d1, cmap=cmap, norm=LogNorm(1e-12, 1e0), interpolation='none') 
        
    rho_e_U1 = U_U1_e.conj().T @ (rho_e) @ U_U1_e
    rho_modes = fn.asymmetry_modes(rho_e_U1, sectors_e)
    print('ENV: ', [f'{omega_idx}: {fn.compute_norm(rho_omega):.2f}' for omega_idx, rho_omega in enumerate(rho_modes)])

    print((fn.renyi_divergence_sym(rho_e, symmetry, 1).real))


In [ ]:
from scipy.special import comb
# given an initial state see how the modes are already populated 

# fig, axs = plt.subplots(len(thetas), N+1, figsize=(22, 16))
# fig, axs = plt.subplots(3, 3, figsize=(22, 16))
renyis = []
for theta_idx, theta in enumerate(thetas[:]):
    print(f'theta: {theta}')

    rho = fn.initial_state(N, sites_to_keep, theta, 'homogenous') # initial_state_test(theta)

    rho_s = fn.ptrace(rho, sites_to_keep)
    rho_e = fn.ptrace(rho, qthrow)

    # rho = U_U1_se.conj().T @ state @ U_U1_se
    # rho_modes = fn.asymmetry_modes(rho, sectors_se)
    # print([f'{omega_idx}: {fn.compute_norm(rho_omega):.2f}' for omega_idx, rho_omega in enumerate(rho_modes)])
    
    rho_s = U_U1_s.conj().T @ rho_s @ U_U1_s
    rho_modes = fn.asymmetry_modes(rho_s, sectors_s)
    print('SYS: ', [f'{omega_idx}: {fn.compute_norm(rho_omega):.3f}' for omega_idx, rho_omega in enumerate(rho_modes)])

    # rho_e = U_U1_e.conj().T @ rho_e @ U_U1_e
    # rho_modes = fn.asymmetry_modes(rho_e, sectors_e)
    # print('ENV:', end=' ')
    # for omega_idx, rho_omega in enumerate(rho_modes):
    #     fn.compute_norm(rho_omega)
    #     print(f'{omega_idx}: {fn.compute_norm(rho_omega):.2f}', end=' ')

        # d2 = np.ma.masked_less(np.abs(rho_omega), 1e-12)
        # im = axs[theta_idx, omega_idx].imshow(d2, cmap=cmap, norm=LogNorm(1e-12, 1e0), interpolation='none')
    # d2 = np.ma.masked_less(np.abs(rho), 1e-12)
    # axs[1, theta_idx].imshow(d2, cmap=cmap, norm=LogNorm(1e-12, 1e0), interpolation='none')
# fig.tight_layout()

In [ ]:
import importlib
import circuit_obj, functions
importlib.reload(circuit_obj)
importlib.reload(functions)
Circuit = circuit_obj.Circuit
fn = functions

In [ ]:
circuits = []
circuit_realizations_max = 100
if symmetry == 'U1':
    try:
        h_list_all = np.load(f'data/U1_rnd_parameters.npy')
        print('Loaded U1 parameters')
    except FileNotFoundError:
        h_list_all = np.random.uniform(-np.pi, np.pi, 5*N*circuit_realizations_max).reshape(circuit_realizations_max, N, 5) /alphaT
        np.save(f'data/U1_rnd_parameters.npy', h_list_all)
        print('Generated U1 parameters')
elif symmetry == 'SU2':
    try:
        h_list_all = np.load(f'data/SU2_rnd_parameters.npy')
    except FileNotFoundError:
        h_list_all = np.random.uniform(-np.pi, np.pi, 1*N*circuit_realizations_max).reshape(circuit_realizations_max, N, 1) /alphaT
        np.save(f'data/SU2_rnd_parameters.npy', h_list_all)
        
snapshots_t = np.array([t for t in [0, 10, 50, 100, 300, 500, 1000] if t <= T])


for circuit_realization in range(circuit_realizations):
    if symmetry == 'U1':
        h_list = h_list_all[circuit_realization]
        gates = [fn.gen_u1([*h]) for h in h_list]
    elif symmetry == 'SU2':
        h_list = h_list_all[circuit_realization]
        gates = [fn.gen_su2(*h) for h in h_list]

    order = fn.gen_gates_order(N, geometry=geometry)    
    circuit = Circuit(N=N, T=T, gates=gates, order=order, symmetry=symmetry)
    circuit.projectors = projectors_s
    circuit.U_U1_s = U_U1_s
    circuit.sectors_s = sectors_s
    circuit.Ns = Ns
    circuit.snapshots_t = snapshots_t
    circuits.append(circuit)
len(circuits)

In [ ]:
def compute_circuit(theta, circuit_real):
    circuit = circuits[circuit_real]
    rho = fn.initial_state(N, sites_to_keep, theta, 'homogenous') # initial_state_test(theta)
    return circuit.run(masks_dict, sites_to_keep, alphas, rho)

if globals().get('renyi') is None or globals(
    ).get('renyi').shape != (circuit_realizations, len(thetas), len(alphas), T + 1):
    renyi = np.zeros((circuit_realizations, len(thetas), len(alphas), T + 1), dtype=np.float64)
    
if globals().get('norms_s') is None or globals(
    ).get('norms_s').shape != (circuit_realizations, len(thetas), T + 1, Ns+1):
    norms_s = np.zeros((circuit_realizations, len(thetas), T + 1, Ns+1), dtype=np.float64)
    
if globals().get('evo') is None or globals(
    ).get('evo').shape != (circuit_realizations, len(thetas), T + 1, 2**N):
    evo = np.zeros((circuit_realizations, len(thetas), T + 1, 2**N), dtype=np.complex128)

for theta_i, circuit_real in tqdm(product(
        range(len(thetas)), #
        range(circuit_realizations), 
    ), total=len(thetas) * circuit_realizations):
    theta = thetas[theta_i]
    print(f'circuit_realization: {circuit_real}, theta: {theta/np.pi:.2f} pi')
    a0, a1, a2 = compute_circuit(theta, circuit_real)
    renyi[circuit_real, theta_i, :, :], norms_s[circuit_real, theta_i, :, :], evo[circuit_real, theta_i, :, :] = a0, a1, a2
    np.savez(f'data/{symmetry}_theta{theta/np.pi:.2f}_circuit_real{circuit_real}_T{T}.npz',
            renyi=a0, 
            norms_s=a1, 
            evo=a2,
            theta=theta,
            circuit_real=circuit_real,
            alphas=alphas,
            T=T,
            Ns=Ns,
            N=N,
        )
    

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 8))
for theta_idx, theta in enumerate(thetas):
    res_ = np.mean(renyi[:, theta_idx, 0, :], axis=0)
    # fn.print_matrix(res[:, theta_idx, 0, :])
    # print()
    ax.plot(res_, label=f'{theta:.2f}', ls='-', marker='.')
    
ax.set_yscale('log')
ax.set_xscale('log')
ax.legend()

In [ ]:
%matplotlib osx 
fig, ax = plt.subplots(1, 1, figsize=(16, 8))

color_ = ax._get_lines.get_next_color()
for mode in range(Ns+1):
    for theta_idx, theta in enumerate(thetas):
        ls_ = ['-','-.',':'][theta_idx%4]
        temp_norm = norms_s[0, theta_idx, :, mode].astype(np.float64) #np.mean(norms_s[:, theta_idx, :, mode].astype(np.float64), axis=0)
        
        ti, tf = 10, T
        t = np.arange(ti, tf)
        ln_t = np.log(t)
        ln_y = np.log(temp_norm[ti:tf])
        
        # do a simple linear regression: ln_y = C - b*ln_t
        A = np.vstack([ln_t, np.ones_like(ln_t)]).T
        slope1, intercept1 = np.linalg.lstsq(A, ln_y, rcond=None)[0]
        ax.plot(t, np.exp(intercept1 + slope1 * ln_t), color=color_, linewidth=4, alpha=.45)
        ax.plot(temp_norm, linestyle=ls_, color=color_, label=rf'$\omega={mode}, \theta={theta/np.pi:.2f}$, b={slope1:.10f}')
        # ax.plot(temp_norm, '.', color=color_,)
        color_ = ax.lines[-1].get_color()
    color_ = ax._get_lines.get_next_color()
ax.set_yscale('log')
ax.set_xscale('log')
ax.legend()

In [ ]:
%matplotlib osx
# fig, axs = plt.subplots(len(snapshots_t[-2:]), len(thetas), figsize=(12, 8))
renyis = []
modes_e = np.zeros((len(thetas), len(snapshots_t), Ne+1))
avg_s = np.zeros((len(thetas), len(snapshots_t), Ns+1))
avg_e = np.zeros((len(thetas), len(snapshots_t), Ne+1))
for theta_idx, theta in enumerate(thetas):
    for t in range(T):
        
        rho_s = np.zeros((2**Ns, 2**Ns), dtype=np.complex128)
        rho_e = np.zeros((2**Ne, 2**Ne), dtype=np.complex128)
        for circuit_real in tqdm(range(circuit_realizations)):
            state = evo[circuit_real,theta_idx,t,:]
            rho_s += fn.ptrace(state, sites_to_keep)
            rho_e += fn.ptrace(state, qthrow)
        rho_s /= circuit_realizations
        rho_e /= circuit_realizations
        rho_s_U1 = U_U1_s.conj().T @ rho_s @ U_U1_s 
        rho_e_U1 = U_U1_e.conj().T @ rho_e @ U_U1_e
        # d2 = np.ma.masked_less(np.abs(rho_s_U1), 1e-12)
        # im = axs[time_idx, theta_idx].imshow(d2, cmap=cmap, norm=LogNorm(1e-12, 1e0), interpolation='none')
        rho_s_modes = fn.asymmetry_modes(rho_s_U1, sectors_s)
        print(theta_idx, t, )
        print('SYS: ', [f'{omega_idx}: {fn.compute_norm(rho_omega):.2f}' for omega_idx, rho_omega in enumerate(rho_s_modes)])
        avg_s[theta_idx, t, :] = [np.sum(np.abs(rho_s_mode)) for rho_s_mode in rho_s_modes]
        rho_e_modes = fn.asymmetry_modes(rho_e_U1, sectors_e)
        modeee = [fn.compute_norm(rho_omega) for omega_idx, rho_omega in enumerate(rho_e_modes)]
        modes_e[theta_idx, t, :] = modeee
        print('ENV: ', [f'{omega_idx}: {fn.compute_norm(rho_omega):.2f}' for omega_idx, rho_omega in enumerate(modeee)])
        avg_e[theta_idx, t, :] = [np.sum(np.abs(rho_e_mode)) for rho_e_mode in rho_e_modes]
        
    
fig.tight_layout()

In [ ]:
cum_sectors_s = np.cumsum(sectors_s)
def get_sector_avg(pts):
    new_pts = np.zeros_like(pts)
    start = 0
    avgs = np.zeros_like(cum_sectors_s, np.float64)
    for sector_idx, end in enumerate(cum_sectors_s):
        temp_avg = np.mean(pts[start:end])
        new_pts[start:end] = temp_avg
        avgs[sector_idx] = temp_avg
        start = end
    return new_pts, avgs

In [ ]:
%matplotlib osx
fig, axs = plt.subplots(3, len(thetas), figsize=(12, 10))
fig, axs1 = plt.subplots(2, 1, figsize=(8, 10))
for yline in cum_sectors_s:
    axs1[0].axvline(yline, color='k', linestyle='--', alpha=.5)
renyis = []
for theta_idx, theta in enumerate(thetas):
    t = 0
    # rho_s = np.zeros((2**Ns, 2**Ns), dtype=np.complex128)
    # for circuit_real in tqdm(range(circuit_realizations)):
    state = evo[circuit_real,theta_idx,t,:]
    rho_s = fn.ptrace(state, sites_to_keep)
    # rho_s /= circuit_realizations
    rho_s_U1 = U_U1_s.conj().T @ rho_s @ U_U1_s 
    # for omega_idx, rho_omega in enumerate([rho_modes[0]]):
    #     d2 = np.ma.masked_less(np.abs(rho_omega), 1e-12)
    #     im = axs[0, theta_idx].imshow(d2, cmap=cmap, norm=LogNorm(1e-12, 1e0), interpolation='none')
    d2 = np.ma.masked_less(np.abs(rho_s_U1), 1e-12)
    im = axs[0, theta_idx].imshow(d2, cmap=cmap, norm=LogNorm(1e-12, 1e0), interpolation='none')
    diag = np.diag(d2)
    diag_avg, avgs = get_sector_avg(diag)
    print(avgs)
    axs1[0].plot(diag_avg, alpha=.5, linewidth=2)
    axs1[0].plot(diag, label=f'{theta:.2f}, t={t}', ls='-')
    ax1s[1].plot(avgs, label=f'{theta:.2f}, t={t}', ls='-')
    
    
    t = 20
    state = evo[circuit_real,theta_idx,t,:]
    rho_s = fn.ptrace(state, sites_to_keep)
    rho_s_U1 = U_U1_s.conj().T @ rho_s @ U_U1_s     
    d2 = np.ma.masked_less(np.abs(rho_s_U1), 1e-12)
    im = axs[1, theta_idx].imshow(d2, cmap=cmap, norm=LogNorm(1e-12, 1e0), interpolation='none')
    color_ = axs1[0].lines[-1].get_color()
    diag = np.diag(d2)
    diag_avg, avgs = get_sector_avg(diag)
    print(avgs)
    axs1[0].plot(diag_avg, alpha=.5, linewidth=2, color=color_)
    axs1[0].plot(diag, label=f'{theta:.2f}, t={t}', ls='--', color=color_)
    ax1s[1].plot(avgs, label=f'{theta:.2f}, t={t}', ls='--', color=color_)
    
    
    t = T-1
    state = evo[circuit_real,theta_idx,t,:]
    rho_s = fn.ptrace(state, sites_to_keep)
    rho_s_U1 = U_U1_s.conj().T @ rho_s @ U_U1_s     
    d2 = np.ma.masked_less(np.abs(rho_s_U1), 1e-12)
    im = axs[2, theta_idx].imshow(d2, cmap=cmap, norm=LogNorm(1e-12, 1e0), interpolation='none')
    diag = np.diag(d2)
    diag_avg, avgs = get_sector_avg(diag)
    print(avgs)
    axs1[0].plot(diag_avg, alpha=.5, linewidth=2, color=color_)
    axs1[0].plot(diag, label=f'{theta:.2f}, t={t}', ls=':', color=color_)
    ax1s[1].plot(avgs, label=f'{theta:.2f}, t={t}', ls=':', color=color_)
    ax1s[0].plot(cum_sectors_s, avgs, label=f'{theta:.2f}, t={t}', ls=':', color=color_, linewidth=2, marker='o')
axs1[0].legend()
axs1[0].set_yscale('log')  
axs1[1].set_yscale('log')  
fig.tight_layout()

In [ ]:
def size_omega_subspace(Ns):
    """
    Calculate the size of the omega subspace for a given number of qubits Ns.
    The size of the omega subspace is defined as the number of ways to choose
    m1 and m2 such that m1 - m2 = omega, where m1 and m2 are the number of
    qubits in the first and second half of the system, respectively.
    """
    # Initialize a dictionary to store the size of the omega subspace
    size_omega = {}

    for m1 in np.arange(-Ns/2, Ns/2+1):
        for m2 in np.arange(-Ns/2, Ns/2+1):
            omega_key = int(m1 - m2)
            # print(m1, m2, end='; ')
            if omega_key not in size_omega:
                size_omega[omega_key] = 0
            size_omega[omega_key] += comb(Ns, int(Ns/2+m1)) * comb(Ns, int(Ns/2+m2))

    # sort the dictionary by key
    sorted_size_omega = dict(sorted(size_omega.items()))
    size_omega = {}
    # print the sorted dictionary
    for key, value in sorted_size_omega.items():
        if key>0:
            # print(f"{key}: {value*2}")
            size_omega[key] = value*2
            
    # convert the dictionary to a numpy array
    size_omega_array = np.array(list(size_omega.values())[::-1], dtype=np.float64)
    # print the numpy array
    return size_omega #size_omega_array

size_omega_subspace(N)[7]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))

state = np.mean(evo[:, 4, T-1, :], axis=0)
rho_s = fn.ptrace(state, sites_to_keep)
rho_s_tw = fn.manual_U1_tw(rho_s, projectors_s)
rho_s_U1 = U_U1_s.conj().T @ (rho_s - rho_s_tw) @ U_U1_s
rho_s_U1 = U_U1_s.conj().T @ (rho_s) @ U_U1_s
rho_s_modes = fn.asymmetry_modes(rho_s_U1, sectors_s)

rho_eq = rho_s_U1.copy()
d1 = np.ma.masked_less(np.abs(rho_s_U1), 1e-12)

rho_eq[rho_eq != 0] = np.mean(d1)


cmap = cm.plasma.copy()
cmap.set_bad(color='white')  # or 'none' for full transparency

ax.imshow(np.abs(d1), cmap=cmap, norm=LogNorm(1e-12, 1e0), interpolation='none') 

from scipy.sparse import csr_matrix
# print(csr_matrix(d1))
np.mean(d1), np.std(d1),  np.max(d1), np.min(d1)

In [ ]:
(0.000715776840252967, 7.666467083416871e-20)
(0.00020219214750874813, 8.464466801715275e-05)
(9.286443172843784e-05, 5.5277308879935246e-05)
(8.18469776915174e-05, 4.591417324268909e-05)
(7.19991515530784e-05, 4.192462579240732e-05)
(5.525441391170834e-05, 3.1446906009458656e-05)
(, 2.4.8989316690781734e-05784783389054931e-05)



In [171]:
norms_s_old = norms_s

In [172]:
import os

# Specify the directory
directory = "results/U1/T10000/"

# List all folders
Big_folders = [f for f in os.listdir(directory) if os.path.isdir(os.path.join(directory, f))]

thetas = np.array([.3, .35, .4, .45, .5])*np.pi # theta

theta_dic = {}
renyis = np.zeros((5, 40, 10001))
norms_s = np.zeros((5, 40, 10001, 9))
for theta_idx, theta in enumerate(thetas):
    for circuit_realization in range(40):
        try:
                
            archive = np.load(directory + f'theta{theta/np.pi:.2f}/circuit_real{circuit_realization}/renyi/data.npz', allow_pickle=None)
            renyi = archive[archive.files[0]]
            first_renyi = renyi[0, 0]
            theta_dic[theta/np.pi] = first_renyi    
            renyis[theta_idx, circuit_realization] = renyi
                
            archive = np.load(directory + f'theta{theta/np.pi:.2f}/circuit_real{circuit_realization}/norms_s/data.npz', allow_pickle=None)
            norms = archive[archive.files[0]]
            first_renyi = norms[0, 0]
            theta_dic[theta/np.pi] = first_renyi    
            norms_s[theta_idx, circuit_realization] = norms
        except FileNotFoundError:
            print(f'file not found: theta{theta/np.pi:.2f}/circuit_real{circuit_realization}')
        
theta_dic

file not found: theta0.35/circuit_real37
file not found: theta0.35/circuit_real38
file not found: theta0.35/circuit_real39
file not found: theta0.40/circuit_real4


{0.3: 1.0000000000000027,
 0.35: 1.000000000000003,
 0.4: 1.000000000000004,
 0.45: 1.000000000000004,
 0.5: 1.000000000000004}

In [168]:
for theta_idx, theta in enumerate(thetas):
    m   = np.ma.masked_equal(renyis[theta_idx, 10:, :], 0)      # mask out all zeros
    renyi = m.mean(axis=0).data            # .data to get a plain ndarray

    plt.plot(renyi, label=f'theta{theta/np.pi:.2f}')
    plt.scatter(0, theta_dic[theta/np.pi])
plt.xscale('symlog')
plt.yscale('log')
plt.legend()

In [153]:
import os, shutil, logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

# Specify the directory
directory = "results_/"

# List all folders
U1_folders = [f for f in os.listdir(directory) if os.path.isdir(os.path.join(directory, f))]

def keys_for_value(d: dict, target):
    return [k for k, v in d.items() if v == target][0]

directory_final = "results/U1/T10000/"

new_theta_dic = {}
print('total folders:', len(U1_folders))
for U1_folder in U1_folders:
    folder_path = os.path.join(directory, U1_folder)
    circuits = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]
    for circuit in circuits:
        circuit_realization = int(circuit.split('circuit_real')[1])
        folder__path = os.path.join(folder_path, circuit)
        archive = np.load(folder__path + f'/renyi/data.npz', allow_pickle=None)
        renyi = archive[archive.files[0]]
        first_renyi = renyi[0, 0]
        # find closest theta to first_renyi
        closest_theta = keys_for_value(theta_dic, first_renyi)
        # print(f'Folder: {U1_folder}, Circuit: {circuit}, First Renyi: {first_renyi}, theta: {closest_theta}')
        new_theta_dic[closest_theta] = new_theta_dic.get(closest_theta, 0) + 1
        
        destination = os.path.join(directory_final, f'theta{closest_theta:.2f}')
        destination_circuit = os.path.join(directory_final, f'theta{closest_theta:.2f}/circuit_real{circuit_realization}')
        # print(f'Copying {folder__path} to {destination}')
        
        files = ['evo', 'norms_s', 'renyi']
        for file in files:
            src_dir = os.path.join(folder__path, f'{file}')
            src_file = os.path.join(src_dir, 'data.npz')
            
            dest_dir = os.path.join(destination_circuit, f'{file}')
            dest_file = os.path.join(dest_dir, 'data.npz')
            
            os.makedirs(dest_dir, exist_ok=True)
            
            shutil.copy2(src_file, dest_file)
            print(f"Copied {src_file} → {dest_file}")
            


total folders: 51
Copied results_/U1-results_3174013398/circuit_real28/evo/data.npz → results/U1/T10000/theta0.50/circuit_real28/evo/data.npz
Copied results_/U1-results_3174013398/circuit_real28/norms_s/data.npz → results/U1/T10000/theta0.50/circuit_real28/norms_s/data.npz
Copied results_/U1-results_3174013398/circuit_real28/renyi/data.npz → results/U1/T10000/theta0.50/circuit_real28/renyi/data.npz
Copied results_/U1-results_3174013398/circuit_real29/evo/data.npz → results/U1/T10000/theta0.50/circuit_real29/evo/data.npz
Copied results_/U1-results_3174013398/circuit_real29/norms_s/data.npz → results/U1/T10000/theta0.50/circuit_real29/norms_s/data.npz
Copied results_/U1-results_3174013398/circuit_real29/renyi/data.npz → results/U1/T10000/theta0.50/circuit_real29/renyi/data.npz
Copied results_/U1-results_3174013398/circuit_real30/evo/data.npz → results/U1/T10000/theta0.50/circuit_real30/evo/data.npz
Copied results_/U1-results_3174013398/circuit_real30/norms_s/data.npz → results/U1/T10000/

In [139]:
new_theta_dic

{0.5: 30, 0.3: 36, 0.45: 30, 0.4: 30, 0.35: 27}

In [50]:
np.mean(renyi[:, 100, 1], axis=0)

0.6684797927149069

In [75]:
thetas = np.array([.3, .35, .4, .45, .5])*np.pi # theta

In [100]:
norms_s = np.zeros((5, 10, 10001, 9), dtype=np.float64)
for theta_idx, theta in enumerate(thetas):
    res = []
    for idx in range(10):
        try:
            archive = np.load(f'results/U1/T10000/theta{theta/np.pi:.2f}/circuit_real{idx}/norms_s/data.npz', allow_pickle=None)
            data = archive[archive.files[0]]
            res.append(data)
        except FileNotFoundError:
            print(f'File not found: results/U1/T10000/theta{theta/np.pi:.2f}/circuit_real{idx}/norms_s/data.npz')
            res.append(data)
            continue
    res = np.array(res)
    print(res.shape)
    norms_s[theta_idx] = res
norms_s.shape

(10, 10001, 9)
(10, 10001, 9)
File not found: results/U1/T10000/theta0.40/circuit_real4/norms_s/data.npz
(10, 10001, 9)
(10, 10001, 9)
(10, 10001, 9)


(5, 10, 10001, 9)

In [175]:
%matplotlib osx
fig, ax = plt.subplots(1, 1, figsize=(16, 8))

ax.axhline(0.63, color='k', linestyle='--', alpha=.5)
ax.axhline(0.63, color='k', linestyle='--', alpha=.5)
ax.axhline(0.35, color='k', linestyle='--', alpha=.5)
ax.axhline(0.16, color='k', linestyle='--', alpha=.5)
ax.axhline(0.06, color='k', linestyle='--', alpha=.5)
ax.axhline(0.0165, color='k', linestyle='--', alpha=.5)
ax.axhline(0.0045, color='k', linestyle='--', alpha=.5)
ax.axhline(0.0009, color='k', linestyle='--', alpha=.5)
ax.axhline(0.0001, color='k', linestyle='--', alpha=.5)


color_ = ax._get_lines.get_next_color()
for mode in range(Ns+1):
    for theta_idx, theta in enumerate(thetas):

        ls_ = ['-', '--', '-.', ':'][theta_idx%4]
        m   = np.ma.masked_equal(norms_s[theta_idx, :, :, mode], 0)      # mask out all zeros
        norms = m.mean(axis=0).data
        plt.plot(norms, label=f'{mode}, theta={theta/np.pi:.2f}', ls=ls_, color=color_, lw=1.2)
        color_ = ax.lines[-1].get_color()
    color_ = ax._get_lines.get_next_color()
ax.legend()
ax.set_xscale('log')
ax.set_yscale('log')


In [177]:
len(np.arange(40, 70, 3))

10

In [ ]:
for theta_idx, theta in enumerate(thetas):
    m   = np.ma.masked_equal(renyis[theta_idx, 10:, :], 0)      # mask out all zeros
    renyi = m.mean(axis=0).data            # .data to get a plain ndarray

    plt.plot(renyi, label=f'theta{theta/np.pi:.2f}')
    plt.scatter(0, theta_dic[theta/np.pi])
plt.xscale('symlog')
plt.yscale('log')
plt.legend()

In [24]:
import numpy as np
import os

def load_local_npy(path):
    """
    Load a single .npy file and return the array.
    """
    arr = np.load(path, mmap_mode=None)   # or mmap_mode='r' for large files
    return arr

def load_local_npz(path):
    """
    Load a .npz archive and return a dict of arrays.
    """
    archive = np.load(path, allow_pickle=False)
    # e.g. archive.files == ['arr_0'] by default, or your keys if you named them
    return {key: archive[key] for key in archive.files}

if __name__ == "__main__":
    folder =f'results/U1/T10000/theta0.30/circuit_real{idx}/evo/'
    for fname in os.listdir(folder):
        path = os.path.join(folder, fname)
        if fname.endswith(".npy"):
            arr = load_local_npy(path)
            print(f"{fname}: shape={arr.shape}, dtype={arr.dtype}")
        elif fname.endswith(".npz"):
            data = load_local_npz(path)
            for key, arr in data.items():
                print(f"{fname}[{key}]: shape={arr.shape}, dtype={arr.dtype}")


data.npz[arr_0]: shape=(7, 1048576), dtype=complex128


In [25]:
arr

array([[ 0.3972346 +0.j        ,  0.20240114+0.j        ,
         0.20240114+0.j        , ...,  0.        +0.j        ,
         0.        +0.j        ,  0.        +0.j        ],
       [ 0.2552262 -0.3043927j ,  0.027083  -0.02397336j,
         0.21565312-0.12957118j, ...,  0.        +0.j        ,
         0.        +0.j        ,  0.        +0.j        ],
       [-0.06926512-0.39114917j, -0.15008132-0.08143564j,
        -0.03898802-0.2077758j , ...,  0.        +0.j        ,
         0.        +0.j        ,  0.        +0.j        ],
       ...,
       [ 0.31343006+0.24404289j,  0.07916119+0.09069941j,
        -0.04066875+0.19665243j, ...,  0.        +0.j        ,
         0.        +0.j        ,  0.        +0.j        ],
       [ 0.37547889+0.12965699j,  0.11441473+0.0683707j ,
         0.02502754+0.21591978j, ...,  0.        +0.j        ,
         0.        +0.j        ,  0.        +0.j        ],
       [-0.19545976-0.34581905j,  0.03746166-0.11054588j,
         0.1706675 -0.15841132